In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.__version__

'1.3.3'

# Procesamiento columna 'Descripción'

In [3]:
excel_file = pd.ExcelFile('./data/raw/actualizacion_acciones_mantenimiento/data.xlsx', engine='openpyxl')
sheets = excel_file.sheet_names
sheets

['Lista de equipos',
 'Historico calif 2_1(10 años)',
 'T-PAP(C)_T-ACE',
 'ARC(I)_ARC(D)',
 'DP',
 'OIL.TAP',
 'DIEL.PF_ DIEL',
 'OIL.CORR',
 'PRO-M',
 'M.OLTC']

In [4]:
import unidecode

def norm_descrpcion(s):
    s = s.lower()
    s = unidecode.unidecode(s)
    return s

converter_descripcion = {}
converter_descripcion['Descripción'] = norm_descrpcion

df_collection = {}
for name in sheets:
    df_collection[name] = pd.read_excel('./data/raw/actualizacion_acciones_mantenimiento/data.xlsx', sheet_name=name, converters=converter_descripcion)

## Separación manual

In [5]:
def separacion_manual(dframe, originales, reemplazos, adiciones):
    df = dframe.copy()
    for i in range(len(originales)):
        index = df['Descripción'] == originales[i]
        if sum(index) > 0:
            instancias = df.loc[index].copy()
            df.loc[index,'Descripción'] = reemplazos[i]
            df = df.append(instancias, ignore_index = True)
            df.loc[len(df) - sum(index):,'Descripción'] = adiciones[i]
    return df




df_collection_separados = {}

excel_file_reemplazos = pd.ExcelFile('./data/raw/actualizacion_acciones_mantenimiento/reemplazos_manual.xlsx')
sheets_reemplazos = excel_file_reemplazos.sheet_names
df_collection_manuales = {}

converter_manuales = {}
converter_manuales['Originales'] = norm_descrpcion
converter_manuales['Reemplazos'] = norm_descrpcion
converter_manuales['Adiciones'] = norm_descrpcion

for name in sheets_reemplazos:#sheets_reemplazos:
    df_collection_manuales[name] = pd.read_excel('./data/raw/actualizacion_acciones_mantenimiento/reemplazos_manual.xlsx', sheet_name = name, converters = converter_manuales)
    index_manuales = ~df_collection_manuales[name].loc[:,'Reemplazos'].isna()
    originales = df_collection_manuales[name].loc[index_manuales, 'Originales'].values
    reemplazos = df_collection_manuales[name].loc[index_manuales, 'Reemplazos'].values
    adiciones = df_collection_manuales[name].loc[index_manuales, 'Adiciones'].values
    df_collection_separados[name] = separacion_manual(df_collection[name], originales, reemplazos, adiciones)

## Normalización por palabras clave

In [6]:
palabras_clave = {}
palabras_clave['revisar/cambiar motoventilador'] = ['motoventilador','ventilador','motoventiladores','ventiladores']
palabras_clave['realizar analisis dga y humedad seguimiento'] = ['dga']
palabras_clave['hacer seguimiento generacion anormal de gases'] = ['generacion','energizacion','gases']
palabras_clave['cambiar silica oltc'] = ['silica']
palabras_clave['cambiar filtro oltc'] = ['filtro']
palabras_clave['realizar secado oltc'] = ['secado']
palabras_clave['realizar muestreo-analisis fsco-qco oltc'] = ['aceite','muestra','fsco','qco']
palabras_clave['realizar regeneramiento aceite aislante'] = ['regeneramiento','regeneracion']
palabras_clave['adicionar irgamet39 a 100ppm'] = ['irgamet','pasivador','pasivar']
palabras_clave['temperatura devenado'] = ['calibrar indicador temperatura devanados']
palabras_clave['efectuar prueba de descargas parciales'] = ['descargas parc']
palabras_clave['seguimiento durante montaje'] = ['durante montaje']
palabras_clave['verificar condicion antes energizar oltc'] = ['energizar']
palabras_clave['realizar prueba estimacion humedad whrt'] = ['estimacion humedad']
palabras_clave['cambiar valvula alivio presion'] = ['alivio presion', 'alivio de presion','valvula alivio','valvula de alivio','valvula de sobre presion','valvula sobre presion','valvula de sobrepresion','valvula sobrepresion']
palabras_clave['presion subita'] = ['cambiar rele presion subita']
palabras_clave['revisar/cambiar contador cambiador tapsbiador'] = ['contador']
df_collection_pc = df_collection_separados.copy()
unicos_pc = {}
index_unicos_pc = {}

for name in sheets_reemplazos:
    unicos = pd.unique(df_collection_separados[name]['Descripción'])
    index_unicos = np.zeros(len(unicos))
    for i in range(len(unicos)):
        index_unicos[i] = i
        for key,value in palabras_clave.items():
            for word in value:
                if word in unicos[i]:
                    df_collection_pc[name] = df_collection_pc[name].replace(unicos[i],key)
                    unicos[i] = key
                    break
    unicos_pc[name] = unicos
    index_unicos_pc[name] = index_unicos

## Aplicando algoritmo de distancia de Levenshtein

### Algoritmo de distancia de Levenshtein

In [7]:
def dist_lev(s1,s2):
    dist_matrix = np.zeros((len(s1) + 1, len(s2) + 1))
    dist_matrix[1:,0] = range(1,len(s1) + 1)
    dist_matrix[0,1:] = range(1,len(s2) + 1)
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            if s1[i - 1] == s2[j - 1]:
                dist_matrix[i,j] = min(dist_matrix[i - 1, j] + 1, 
                                    dist_matrix[i - 1, j - 1], 
                                    dist_matrix[i, j - 1] + 1)
            else:
                dist_matrix[i,j] = min(dist_matrix[i - 1, j] + 1, 
                                    dist_matrix[i - 1, j - 1] + 1, 
                                    dist_matrix[i, j - 1] + 1)
    return dist_matrix[-1,-1]

### Aplicando y ordenando según distancia

In [8]:
# Extrayendo valores únicos:
ordenados_collection = {}
dist_ordenados_collection = {}
index_ordenados_collection = {}
for name in sheets_reemplazos:
    unicos = pd.unique(unicos_pc[name])
    #Construyendo matriz de distancia de Levenshtein
    dist_unicos = np.zeros((len(unicos), len(unicos)))
    for i in range(len(unicos)):
        for j in range(i + 1, len(unicos)):
            dist_unicos[i, j] = dist_lev(unicos[i], unicos[j])
            dist_unicos[j,i] = dist_unicos[i,j] 
    #Inizializar varibales
    # dist_unicos = np.array([[0,8,8,10.5],[1,0,4,3],[2,5,0,6],[8,10,7,0]])
    # unicos = [1,2,3,4]
    ordenados = ['']*len(unicos)
    dist_ordenados = np.zeros(len(unicos))
    index_ordenados = np.zeros(len(unicos))
    #Ubicando primer elemento
    max_dist_index = np.argmax(dist_unicos)
    max_dist_index = np.unravel_index(max_dist_index, np.shape(dist_unicos))
    row = max_dist_index[0]
    ordenados[0] = unicos[row]
    index_ordenados[0] = row
    dist_ordenados[0] = 0
    dist_unicos[row,:] = np.inf
    #Diagonal igual a infinito
    for i in range(len(unicos)):
        dist_unicos[i,i] = np.inf
    #Ubicando el resto de los elementos
    for i in range(len(unicos) - 1):
        min_dist = np.min(dist_unicos[:,row])
        dist_ordenados[i + 1] = dist_ordenados[i] + min_dist
        row = np.argmin(dist_unicos[:,row])
        ordenados[i + 1] = unicos[row]
        index_ordenados[i + 1] = row
        dist_unicos[row,:] = np.inf
    #Ubicándolos por hoja
    ordenados_collection[name] = ordenados
    dist_ordenados_collection[name] = dist_ordenados
    index_ordenados_collection[name] = index_ordenados

### Escribiendo los resultados en Excel

In [9]:
import xlsxwriter
workbook = xlsxwriter.Workbook('./data/processed/actualizacion_acciones_mantenimiento/unicos_ordenados.xlsx')
for name in sheets_reemplazos:
    worksheet = workbook.add_worksheet(name)
    for i in range(len(ordenados_collection[name])): 
        worksheet.write(i,0,ordenados_collection[name][i])
        worksheet.write(i,1,ordenados_collection[name][i])
        worksheet.write(i,2,dist_ordenados_collection[name][i])
        worksheet.write(i,3,index_ordenados_collection[name][i])

workbook.close()

# Separación por equipos

In [10]:
equipos_unicos = pd.unique(df_collection['Lista de equipos']['Equipo'])
df_collection_equipos = {}
for sheet in sheets_reemplazos:
    df_collection_equipos[sheet] = pd.DataFrame()
    equipos_unicos = pd.unique(df_collection_pc[sheet]['Equipo'])
    for equipo in equipos_unicos:
        df_equipo = df_collection_pc[sheet].loc[df_collection_pc[sheet]['Equipo'] == equipo].copy()
        descripciones_unicas = pd.unique(df_equipo['Descripción'])
        equipo_descripcion = {'Equipo':[equipo]*len(descripciones_unicas),'Descripción':descripciones_unicas}
        df_collection_equipos[sheet] = df_collection_equipos[sheet].append(pd.DataFrame(equipo_descripcion),ignore_index = True)

In [11]:
k = list(df_collection_equipos.keys())

In [12]:
df = pd.DataFrame(columns=['Equipo','Descripción','Medida'])
for i in range(len(k)):
    prov = pd.DataFrame.from_dict(df_collection_equipos[k[i]])
    prov['Medida'] = k[i]
    df = df.append(prov)
df


,Equipo,Descripción,Medida
0,809578,realizar analisis dga y humedad seguimiento,T-PAP(C)_T-ACE
1,809578,revisar/cambiar motoventilador,T-PAP(C)_T-ACE
2,809578,efectuar seguimiento anomalia termica,T-PAP(C)_T-ACE
3,809578,realizar muestreo-analisis fsco-qco oltc,T-PAP(C)_T-ACE
4,810449,realizar analisis dga y humedad seguimiento,T-PAP(C)_T-ACE
...,...,...,...
72,201582,realizar muestreo-analisis fsco-qco oltc,M.OLTC
73,201582,cambiar filtro oltc,M.OLTC
74,200646,corregir pendiente en oltc,M.OLTC
75,200646,realizar evaluacion condicion aceit oltc,M.OLTC


In [13]:
df[df['Equipo'] == 809578]

,Equipo,Descripción,Medida
0,809578,realizar analisis dga y humedad seguimiento,T-PAP(C)_T-ACE
1,809578,revisar/cambiar motoventilador,T-PAP(C)_T-ACE
2,809578,efectuar seguimiento anomalia termica,T-PAP(C)_T-ACE
3,809578,realizar muestreo-analisis fsco-qco oltc,T-PAP(C)_T-ACE
0,809578,realizar analisis dga y humedad seguimiento,ARC(I)_ARC(D)
1,809578,efectuar prueba de descargas parciales,ARC(I)_ARC(D)
74,809578,realizar analisis dga y humedad seguimiento,OIL.TAP
75,809578,adecuar tuberia toma muestreo oltc,OIL.TAP
76,809578,realizar muestreo-analisis fsco-qco oltc,OIL.TAP
77,809578,revisar senalizacion del oltc.,OIL.TAP


In [16]:
import pyodbc
from sqlalchemy import create_engine
from urllib.parse import quote_plus

server = 'compartidos.database.windows.net'
database = 'DB_SQL_RECSYS'
username = 'sant_vel'
password = 'Conexiones2030'   
driver= '{ODBC Driver 17 for SQL Server}'

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
#Truncate the table
cursor.execute("TRUNCATE TABLE acciones_por_equipo;")

# Insert Dataframe into SQL Server:
for index, row in df.iterrows():

    cursor.execute("INSERT INTO acciones_por_equipo (id,accion,medida) values(?,?,?)", row['Equipo'], row['Descripción'], row['Medida'])
cnxn.commit()
cursor.close()